In [37]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [38]:
api_wrapper = WikipediaAPIWrapper(top_k=2, doc_content_chars_max=300)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [39]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import VoyageEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os
load_dotenv()

True

In [40]:
loader = WebBaseLoader("https://docs.langchain.com/oss/python/langchain/agents")
Website_docs = loader.load()
website_Splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(Website_docs)
os.environ["VOYAGE_API_KEY"]  

voyage_embeddings = VoyageEmbeddings(
    model="voyage-2",         
    batch_size=32             
)
vectorstore = FAISS.from_documents(
    website_Splitter,
    voyage_embeddings,
)
vectorstore.save_local("faiss_db")


Retrying langchain_community.embeddings.voyageai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised SSLError: HTTPSConnectionPool(host='api.voyageai.com', port=443): Max retries exceeded with url: /v1/embeddings (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1032)'))).


In [41]:
vectorstore_retriever = vectorstore.as_retriever()

In [42]:
from langchain_core.tools import Tool

retriever = vectorstore_retriever

def summarize_docs(query: str) -> str:
    docs = retriever.invoke(query)
    if not docs:
        return "No relevant context found."
    chunks = [f"Source {i+1}:\n{doc.page_content}" for i, doc in enumerate(docs[:4])]
    return "\n\n".join(chunks)

langchain_agents_tool = Tool(
    name="Langchain_agents",
    description="Answer questions about LangChain agents docs.",
    func=summarize_docs,
)
langchain_agents_tool.name

'Langchain_agents'

In [43]:
# A website where all the research papers are published is arxiv. We can use arxiv API to fetch the research papers and create a tool for it. 
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun 
arxiv_api_wrapper = ArxivAPIWrapper(top_k=2, doc_content_chars_max=300)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)

In [44]:
tools = [wiki_tool, langchain_agents_tool, arxiv_tool]

In [45]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\HP\\.conda\\envs\\myenv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=300)),
 Tool(name='Langchain_agents', description='Answer questions about LangChain agents docs.', func=<function summarize_docs at 0x0000018EEC82D120>),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=300))]

# AGENTS

In [46]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model = "arcee-ai/trinity-large-preview:free",
    temperature = 0.9,
    base_url = "https://openrouter.ai/api/v1",
    default_headers={
        "HTTP-Referer": "http://localhost:8000",
        "X-Title": "RAG with Langchain"
    }
)

In [47]:
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful assistant who answers questions with the help of tools. "
     "You have access to the following tools:\n{tools}\n"
     "If you still don't know the answer after using the tools, say \"I don't know\"."
    ),
    ("human", "{input}"),
])

In [ ]:
from langchain.agents import create_agent
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=(
        "You are a helpful assistant who answers questions with the help of tools. "
        "You have access to the following tools:\n{tools}\n"
        "If you still don't know the answer after using the tools, say \"I don't know\"."
    ),
)

In [53]:
Query = input("Enter your Query")
inputs = {"messages": [{"role": "user", "content": Query}]}
result = agent.invoke(inputs)
print(result)

{'messages': [HumanMessage(content='when was attention all you need paper realeased and what was that about ', additional_kwargs={}, response_metadata={}, id='eda98ed1-a4ca-4b10-aa88-a65e2f0577eb'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 353, 'total_tokens': 377, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'cost': 0, 'is_byok': False, 'cost_details': {'upstream_inference_cost': 0, 'upstream_inference_prompt_cost': 0, 'upstream_inference_completions_cost': 0}}, 'model_provider': 'openai', 'model_name': 'arcee-ai/trinity-large-preview:free', 'system_fingerprint': None, 'id': 'gen-1770578825-5O2oVxnntpoUXkeuQKjM', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c3eb8-b5e8-7a41-9b9a-d216a3238962-0', tool_calls=[{'

In [54]:
print(result["messages"][-1].content)

The paper "Attention Is All You Need" was released in 2017. It was authored by eight scientists working at Google and introduced a new deep learning architecture known as the transformer. This architecture is based on the attention mechanism proposed in 2014 by Bahdanau et al.
